### [1] Module, Variables

In [3]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC  # noqa

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


In [4]:
dict_gAccounts = {
    'kjaehw0207': 'flapaqj1!%',
    'woghks.study': 'ans_entrance00'
}

account_1 = list(dict_gAccounts.keys())[0]
password_1 = dict_gAccounts[account_1]

account_2 = list(dict_gAccounts.keys())[1]
password_2 = dict_gAccounts[account_2]


### [2] Access, Sign-in

In [ ]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [ ]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
btn_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
btn_sign_in.click()

# 계정 이메일 입력
input_account = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
input_account.send_keys(account_1)

# [다음] 버튼 클릭해 패스워드 입력
btn_next_password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
btn_next_password.click()

# 계정 비밀번호 입력
input_password = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
input_password.send_keys(password_1)

# [다음] 버튼 입력해 로그인 클릭
btn_next_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
btn_next_sign_in.click()


In [ ]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지
try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


맞춤 페이지 없음. 시청기록 수집을 시작합니다.


### [3] Collection

#### 1) Functions & Variables

In [ ]:
def get_day(weekday=datetime.today().weekday()):
    list = ['월', '화', '수', '목', '금', '토', '일']
    return list[weekday]


def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d').strftime('%Y-%m-%d')


In [ ]:
# 항목별 Class 이름


# 시청일자 - 목록 content에서 [data-date] 값 추출
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>

day_today = get_day()
print(day_today)


토


#### 2) DataFrame

In [5]:
import pandas as pd
columns_rec = ['title', 'channel', 'running_time',
               'watch_date', 'watch_time', 'category', 'watched_YN', 'account']
df_record = pd.DataFrame(columns=columns_rec)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category,watched_YN,account


In [6]:
columns_ser = ['searchword', 'search_date', 'search_time', 'account']
df_search = pd.DataFrame(columns=columns_ser)
df_search.head()


,searchword,search_date,search_time,account


#### 3) Web Data Collecting

In [ ]:
# 목표 갯수만큼 시청 목록(레코드) 호출

num_target_data = 200
# 개인 활동량 기준 하루에 200개 이상 레코드는 없다고 판단
# 필요 시 input으로 레코드 개수를 추가하거나
# 코드 상 data-date가 오늘이 아닌 걸 기준으로 끊을 수도 있다 -> 이 방법을 활용해봐야

num_press_end = int(num_target_data / 100)

for n in range(num_press_end - 1):
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(2) # 전체 페이지가 뜨도록 Wait

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents = soup.find_all('c-wiz', class_='xDtZAf')
print(f'호출된 레코드 수: {len(contents)}')


호출된 레코드 수: 291


In [ ]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = 0  # df_record의 idx
idx_ser = 0  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime): # 영상 길이가 있는 경우 - 실제 시청기록
            print(f"{idx}: Watch Record")

            title = content.find('a', class_=nm_class_title).text
            category = '' # input(f'{title}의 Category는? \t') 
            # 영상 분류 Model 활용을 위해

            channel = content.find('div', class_=nm_class_channel).text 
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            
            watched_YN = '' # input(f'{title}을 끝까지 시청하셨나요? \t') 
            # 시청여부 구분 함수

            df_record.loc[idx_rec] = [title, channel,
                                      running_time, watch_date, watch_time, category, watched_YN, account_1]
            idx_rec += 1

        else:
            print(f"{idx}: Community Post \t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time, account_1]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")

            num_ad += 1


0: Watch Record
1: Watch Record
2: Watch Record
3: Ad
4: Ad
5: Watch Record
6: Watch Record
7: Ad
8: Watch Record
9: Watch Record
10: Community Post 	 Watch time: 오후 8:35
11: Watch Record
12: Watch Record
13: Watch Record
14: Watch Record
15: Watch Record
16: Watch Record
17: Watch Record
18: Watch Record
19: Watch Record
20: Watch Record
21: Watch Record
22: Watch Record
23: Watch Record
24: Watch Record
25: Watch Record
26: Watch Record
27: Watch Record
28: Watch Record
29: Watch Record
30: Watch Record
31: Watch Record
32: Watch Record
33: Watch Record
34: Watch Record
35: Watch Record
36: Watch Record
37: Watch Record
38: Watch Record
39: Watch Record
40: Watch Record
41: Watch Record
42: Watch Record
43: Watch Record
44: Watch Record
45: Watch Record
46: Watch Record
47: Community Post 	 Watch time: 오후 1:50
48: Watch Record
49: Watch Record
50: Watch Record
51: Watch Record
52: Watch Record
53: Watch Record
54: Watch Record
55: Watch Record
56: Watch Record
57: Watch Record
58: Wa